# PyTorch 自定义 CUDA 算子——以矩阵乘法为例

根据 PyTorch 官网的 [Tutorial](https://pytorch.org/tutorials/advanced/cpp_extension.html)，自定义 CUDA 算子的过程可以分为三步：

    1. 首先在 C++ 文件中定义将要从 Python 调用的函数，并用 pybind11 将这些函数绑定到 Python；此外，这个文件还声明了定义在 CUDA 文件中的函数，C++ 函数进行一些类型检查后将其调用转发给 CUDA 函数。
    2. 在 CUDA 文件中编写实际的 CUDA 内核。
    3. 使用 PyTorch 的 `cpp_extension` 工具编译，以便从 Python 调用。

## 编写 C++ 文件

首先我们编写 C++ 文件 `matmul_cuda.cpp` 如下，它有固定的模板，主要实现检查和转发到定义在 CUDA 文件中的函数的功能。

```c++
#include <torch/extension.h>

// CUDA forward declarations
torch::Tensor matmul_cuda(torch::Tensor a, torch::Tensor b);

// C++ interface
#define CHECK_CUDA(x) TORCH_CHECK(x.type().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)

torch::Tensor matmul(torch::Tensor a, torch::Tensor b) {
    CHECK_INPUT(a);
    CHECK_INPUT(b);

    return matmul_cuda(a, b);
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("matmul_global_memory", &matmul, "MatMul implemented by CUDA.");
}
```

## 编写 CUDA 文件

我们要实现的核心功能需要在 CUDA 文件 `matmul_cuda_kernel.cu` 中实现，CUDA 部分与传统的 CUDA 编程类似，最主要的是需要实现两个函数：一个函数执行我们不希望显式手动编码和调用 CUDA 内核的操作，另一个是实际的 CUDA 内核，用于我们想加速的部分。其中第一个函数如下所示：

```c++
torch::Tensor matmul_cuda(torch::Tensor a, torch::Tensor b) {
    const int bs = a.size(0);
    const int h = a.size(1);

    // the tensor a is of size `(bs, h, ma, ka)`
    const int ma = a.size(-2);
    const int ka = a.size(-1);

    // the tensor b is of size `(bs, h, kb, nb)`
    const int kb = b.size(-2);
    const int nb = b.size(-1);

    if (ka != kb) {
        throw std::invalid_argument("Size of tensor A must match size of tensor B.");
    }

    // configure cuda
    const int threads = 32;
    const dim3 threads_per_block(threads, threads, 1);
    const dim3 blocks_per_grid(ma / threads + 1, nb / threads + 1, bs);

    auto tensor_options = torch::TensorOptions().dtype(a.dtype()).device(torch::kCUDA, a.device().index());
    auto out = torch::zeros({bs, h, ma, nb}, tensor_options);

    AT_DISPATCH_FLOATING_TYPES_AND_HALF(
        a.type(), "matmul_cuda", ([&] {
            matmul_cuda_kernel<scalar_t><<<blocks_per_grid, threads_per_block>>>(
                a.packed_accessor32<scalar_t, 4, torch::RestrictPtrTraits>(),
                b.packed_accessor32<scalar_t, 4, torch::RestrictPtrTraits>(),
                out.packed_accessor32<scalar_t, 4, torch::RestrictPtrTraits>(), h, ma, nb, ka);
        }));
    return out;
}
```

其中，`AT_DISPATCH_FLOATING_TYPES_AND_HALF` 宏接受一个类型（本例中为 `a.type()`），一个名称（用于错误消息）和一个 lambda 函数。在这个 lambda 函数中，类型 `scalar_t` 被定义为该上下文中的 Tensor 在实际运行时的类型，因此可以同时适用于多种数据类型。内核启动部分（`<<<...>>>`）与传统 CUDA 编程一致。所有的运算可以看作一个 grid，每个 grid 可以分成若干 block，每个 block 又可以含有若干 thread。在本例的矩阵乘法计算中，每个 thread 计算结果 `c` 中的一个元素的值，具体的计算过程定义在 CUDA 内核函数中，如下所示：

```c++
template <typename scalar_t>
__global__ void matmul_cuda_kernel(
    const torch::PackedTensorAccessor32<scalar_t, 4, torch::RestrictPtrTraits> a,
    const torch::PackedTensorAccessor32<scalar_t, 4, torch::RestrictPtrTraits> b,
    torch::PackedTensorAccessor32<scalar_t, 4, torch::RestrictPtrTraits> out,
    const int num_heads,
    const int m,
    const int n,
    const int k) {
    const int bidx = blockIdx.z;
    const int row = blockIdx.x * blockDim.x + threadIdx.x;
    const int col = blockIdx.y * blockDim.y + threadIdx.y;
    if (row < m && col < n) {
        for (int hidx = 0; hidx < num_heads; hidx++) {
            scalar_t val = 0.0;
            for (int i = 0; i < k; i++) {
                val += a[bidx][hidx][row][i] * b[bidx][hidx][i][col];
            }
            out[bidx][hidx][row][col] = val;
        }
    }
}
```

其中 Accessor 的使用增强了代码的可读性，让我们可以按照 `a[bidx][hidx][row][i]` 这样的格式访问 Tensor 中的元素，而不用显式地指明 stride。`torch::PackedTensorAccessor32` 的作用是指明产生带有32位整数索引的 Packed accessor （若采用64位整数索引则性能会显著降低）。调用时的 `scalar_t` 指明数据类型，`4` 指数据的维数，`torch::RestrictPtrTraits` 表明必须使用 `__restrict__` 关键字。

### JIT 编译扩展及在 Python 中调用

PyTorch 的 JIT 编译机制提供了一种动态编译和加载扩展的方法，只需如下代码即可实现：

In [1]:
import torch
from torch.utils.cpp_extension import load

matmul_cuda = load(
    name='matmul_cuda', 
    sources=["src/matmul_cuda.cpp", "src/matmul_cuda_kernel.cu"]
)

注意此处生成的 Python 模块与 `setuptools` 生成的模块完全相同，但不需要维护一个单独的 `setup.py` 文件。第一次运行时会花费一些时间，因为扩展在后台编译，但由于采用Ninja构建系统，因此编译是增量式的，第二次运行时重新加载扩展会很快。

下面我们验证一下实现的 CUDA 矩阵乘法与 PyTorch 内置的 `matmul` 函数（采用 cuBlas）在精度与速度上的差距：

首先初始化两个矩阵，尺寸分别为 `(8, 4, 128, 32)` 和 `(8, 4, 32, 128)`

In [2]:
a = torch.rand((8, 4, 128, 32), device="cuda")
b = torch.rand((8, 4, 32, 128), device="cuda")

In [3]:
%timeit torch.matmul(a, b)

18.6 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
%timeit matmul_cuda.matmul(a, b)

184 µs ± 2.71 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
torch_out = torch.matmul(a, b)
cuda_out = matmul_cuda.matmul(a, b)
max_err = ((torch_out - cuda_out).abs() / torch_out).max()
mean_err = ((torch_out - cuda_out).abs() / torch_out).mean() 
print(f"max_error: {max_err}, mean_error: {mean_err}")

max_error: 0.0002849252778105438, mean_error: 4.937971607432701e-05


本实验的运行环境为 Ubuntu 20.04 + CUDA 11.3 + PyTorch 1.10，硬件为 NVIDIA RTX 3090 显卡 + AMD 5950X CPU 的组合。从以上结果可以看出，我们自己实现的矩阵乘法 CUDA 算子的平均单次运行时间为 184 微秒，与 PyTorch 的 cuBlas 实现（18.6 微秒）相比，慢了大概10倍，平均误差在 5e-5 左右，最大误差在 3e-4 左右。

## 性能调优：利用 CUDA shared memory

优化 CUDA 代码首先考虑的就是利用 shared memory。在前文所述的实现中，每个 CUDA thread 直接从 global memory 中读取数据再进行计算，代价很大（通常需要几百个时钟周期），而如果首先将一个 block 内需要用的数据搬运到 shared memory（block 内所有 thread 共享的内存），再访问就能显著降低代价（通常为几十个时钟周期）。只需修改 `matmul_cuda_kernel.cu` 内的 CUDA 内核即可：

```c++
template <typename scalar_t>
__global__ void matmul_shared_memory_cuda_kernel(
    const torch::PackedTensorAccessor32<scalar_t, 4, torch::RestrictPtrTraits> a,
    const torch::PackedTensorAccessor32<scalar_t, 4, torch::RestrictPtrTraits> b,
    torch::PackedTensorAccessor32<scalar_t, 4, torch::RestrictPtrTraits> out,
    const int num_heads,
    const int m,
    const int n,
    const int k,
    const int block_size) {
    const int bidx = blockIdx.z;
    const int row = blockIdx.x * blockDim.x + threadIdx.x;
    const int col = blockIdx.y * blockDim.y + threadIdx.y;

    extern __shared__ char data[];
    scalar_t* _sh_tile_a = (scalar_t*)data;
    scalar_t* _sh_tile_b = (scalar_t*)(block_size * block_size * sizeof(scalar_t) + data);

    const int num_blocks = (k + block_size - 1) / block_size;

    if (row < m && col < n) {
        for (int hidx = 0; hidx < num_heads; hidx++) {
            scalar_t val = 0.0;
            for (int i = 0; i < num_blocks; i++) {
                _sh_tile_a[threadIdx.x * block_size + threadIdx.y] = a[bidx][hidx][row][i * block_size + threadIdx.y];
                _sh_tile_b[threadIdx.x * block_size + threadIdx.y] = b[bidx][hidx][i * block_size + threadIdx.x][col];
                __syncthreads();

                for (int j = 0; j < block_size; j++) {
                    val += _sh_tile_a[threadIdx.x * block_size + j] * _sh_tile_b[j * block_size + threadIdx.y];
                }
                __syncthreads();
            }
            out[bidx][hidx][row][col] = val;
        }
    }
}
```

启动此 CUDA 内核时需要相应修改为

```c++
AT_DISPATCH_FLOATING_TYPES_AND_HALF(
        a.type(), "matmul_shared_memory_cuda", ([&] {
            matmul_shared_memory_cuda_kernel<scalar_t>
                <<<blocks_per_grid, threads_per_block, 2 * block_size * block_size * sizeof(scalar_t)>>>(
                    a.packed_accessor32<scalar_t, 4, torch::RestrictPtrTraits>(),
                    b.packed_accessor32<scalar_t, 4, torch::RestrictPtrTraits>(),
                    out.packed_accessor32<scalar_t, 4, torch::RestrictPtrTraits>(), h, ma, nb, ka, block_size);
        }));
```

注意 kernel function 中 `__shared__` 关键字的作用就是声明此处数据在 shared memory 中，与 `<<<...>>>` 中的第三个参数 `2 * block_size * block_size * sizeof(scalar_t)` 一起使用，作用是在运行时根据 `block_size` 的大小动态申请 shared memory。`extern __shared__ char data[];` 此句为固定写法，必须为 `char` 类型，`data` 的大小即为 `<<<...>>>` 中的第三个参数，然后通过指针的类型转换将申请到的 shared memory 空间分配给不同的变量 `_sh_tile_a` 和 `_sh_tile_b`。

下面我们再来测试一下性能：

In [6]:
a = torch.rand((8, 4, 128, 32), device="cuda")
b = torch.rand((8, 4, 32, 128), device="cuda")

In [7]:
%timeit matmul_cuda.matmul_shared_memory(a, b, 16)

61.8 µs ± 3.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
torch_out = torch.matmul(a, b)
cuda_out = matmul_cuda.matmul_shared_memory(a, b, 16)
max_err = ((torch_out - cuda_out).abs() / torch_out).max()
mean_err = ((torch_out - cuda_out).abs() / torch_out).mean() 
print(f"max_error: {max_err}, mean_error: {mean_err}")

max_error: 0.00032089799060486257, mean_error: 4.854623330174945e-05


可以看到采用 shared memory 后，在计算精度基本保持不变的情况下，速度由原来的 180 微秒变为 60 微秒，加速了 3 倍左右。